In [1]:
import os
import tensorflow as tf
import numpy as np

In [3]:
from Packages.NoveltyDetection.VariationalAutoencoders.models.vae import VariationalAutoencoder

In [4]:
# MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

image_size = x_train.shape[1]
original_dim = image_size * image_size
x_train = np.reshape(x_train, [-1, original_dim])
x_test = np.reshape(x_test, [-1, original_dim])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# network parameters
input_shape = (original_dim,)
intermediate_dim = 512
batch_size = 128
latent_dim = 2
epochs = 50

model = VariationalAutoencoder(original_dim, intermediate_dim, latent_dim)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer, loss=tf.keras.losses.mse)
model.fit(x_train, x_train, epochs=5, batch_size=batch_size)

Train on 60000 samples
Epoch 1/5
  128/60000 [..............................] - ETA: 8s

TypeError: 'list' object is not callable

In [65]:
model.predict(x_test)

array([[0.0029497 , 0.00308441, 0.00341599, ..., 0.00347914, 0.00260501,
        0.00285577],
       [0.00454369, 0.00437536, 0.00416983, ..., 0.00404289, 0.00411001,
        0.00410725],
       [0.00417392, 0.00447969, 0.00478648, ..., 0.00481452, 0.00410879,
        0.0043331 ],
       ...,
       [0.00499117, 0.00495851, 0.00476878, ..., 0.0047059 , 0.00490871,
        0.00487351],
       [0.00479537, 0.00492837, 0.00497764, ..., 0.00490778, 0.00462991,
        0.00475683],
       [0.00490433, 0.00493783, 0.00488037, ..., 0.00479532, 0.00469183,
        0.00476579]], dtype=float32)

In [68]:
model.encoder.get_input_at('z_mean')

RuntimeError: The layer has never been called and thus has no defined input.

In [42]:
def vae_loss(y_true, y_pred):
    reconstruction_loss = tf.keras.losses.mse(y_true, y_pred)
    reconstruction_loss *= original_dim
    
    z_mean, z_log_var, z = model.get_output_at('encoder')
    # z_log_var = model.model.encoder.get_output_at('z_log_var')
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    kl = z_mean + tf.exp(0.5 * z_log_var) * epsilon
    return tf.mean(kl, reconstruction_loss)

In [29]:
def plot_results(models,
                 data,
                 batch_size=128,
                 model_name="vae_mnist"):
    """Plots labels and MNIST digits as function of 2-dim latent vector

    # Arguments
        models (tuple): encoder and decoder models
        data (tuple): test data and label
        batch_size (int): prediction batch size
        model_name (string): which model is using this function
    """

    encoder, decoder = models
    x_test, y_test = data
    os.makedirs(model_name, exist_ok=True)

    filename = os.path.join(model_name, "vae_mean.png")
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(x_test,
                                   batch_size=batch_size)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y_test)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.savefig(filename)
    plt.show()

    filename = os.path.join(model_name, "digits_over_latent.png")
    # display a 30x30 2D manifold of digits
    n = 30
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = digit

    plt.figure(figsize=(10, 10))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap='Greys_r')
    plt.savefig(filename)
    plt.show()

In [27]:
models = (model.encoder, model.decoder)
data = (x_test, y_test)


In [32]:
encoder = vae.encoder

In [33]:
encoder.get_output_at

<bound method Layer.get_output_at of <Packages.NoveltyDetection.VariationalAutoencoders.models.vae.Encoder object at 0x7fab3d7bcb10>>

In [30]:
plot_results(models,
             data,
             batch_size=batch_size,
             model_name="vae_mlp")

AttributeError: 'Encoder' object has no attribute 'predict'